## Create "Model Topper" using example

In [18]:

import os

import numpy as np
import pandas as pd

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras import applications

from keras import backend as K

In [9]:
# dimensions of our images.
img_width, img_height = 299, 299

# dimensions of our images.
img_width, img_height = 299, 299
n_features = 8

top_model_weights_path = 'bottleneck_xception_model.h5'
train_data_dir = 'multi_class_testing/train/'
validation_data_dir = 'multi_class_testing/test/'

available_train_files = len(os.listdir(train_data_dir + 'female/')) \
    + len(os.listdir(train_data_dir + 'male/'))
available_test_files = len(os.listdir(validation_data_dir + 'female/')) \
    + len(os.listdir(validation_data_dir+'male/'))
    
nb_train_samples  = available_train_files - available_train_files % 8
nb_validation_samples = available_test_files - available_test_files % 8

batch_size = 8

__Set up Multiclass Testing__

In [10]:
watch_df = pd.read_csv('all_watch_info_with_indicators.csv')
watch_dict = dict()

Replace These Features with something better

In [11]:
for image_name in watch_df['new_image_name']:
    watch_dict[image_name+'.jpg'] =  watch_df[
        watch_df['new_image_name'] == image_name][
            ['is_gold','is_silver','is_strap',
             'is_rotating','is_chronograph',
             'is_square','is_round','is_male']].values[0]
    

Multiclass Helper Functions

In [12]:
def multiclasses_getter(x, i, gen):
    count = 0
    y = np.zeros((batch_size, n_features), dtype=np.int8)
    
    # Feature not working
    if np.shape(x)[0] < gen.batch_size:
        steps = np.shape(x)[0]
    else:
        steps = gen.batch_size
        
    idx = (i) * gen.batch_size
    
    for f in gen.filenames[idx : idx + steps]:
        k = f.split('/')[-1]
        y[count,:] = watch_dict[k] 
        count += 1
    return y

In [13]:
def multiclass_flow_train(flow_from_directory_gen, multiclasses_getter):
    for i, (x, y) in enumerate(flow_from_directory_gen):
        yield x, multiclasses_getter(x, i,train_generator)
    
def multiclass_flow_test(flow_from_directory_gen, multiclasses_getter):
    for i, (x, y) in enumerate(flow_from_directory_gen):
        yield x, multiclasses_getter(x, i, test_generator)
      

Create Multi-lable metrics (Recovered from Keras 1.x)

In [19]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

Create Generators

In [14]:
datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=90,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=False)

test_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    shuffle=False)

Found 7368 images belonging to 2 classes.
Found 3152 images belonging to 2 classes.


Load Standard Xception Model

In [8]:
model = applications.xception.Xception(include_top=False, weights='imagenet')

Build My "Custom" Model

In [9]:
watch_model = Sequential()
watch_model.add(model)
watch_model.add(GlobalAveragePooling2D(name='avg_pool'))
watch_model.add(Dense(n_features, activation="sigmoid"))
watch_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
avg_pool (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 16392     
Total params: 20,877,872
Trainable params: 20,823,344
Non-trainable params: 54,528
_________________________________________________________________


Freeze Exception Layers

In [15]:
for layer in watch_model.layers[0].layers:
    layer.trainable = False

Compile

In [20]:
watch_model.compile(
    loss = "binary_crossentropy", 
    optimizer='sgd', 
    metrics=["binary_accuracy",precision,recall])

Fit 

In [22]:
epochs = 10

In [21]:
# fine-tune the model
watch_model.fit_generator(
        multiclass_flow_train(train_generator, multiclasses_getter),
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = multiclass_flow_test(test_generator, multiclasses_getter),
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/2
921/921 [==============================] - 452s - loss: 0.5002 - binary_accuracy: 0.7815 - precision: 0.7274 - recall: 0.6333 - val_loss: 0.5880 - val_binary_accuracy: 0.7540 - val_precision: 0.6988 - val_recall: 0.7675
Epoch 2/2
921/921 [==============================] - 451s - loss: 0.0866 - binary_accuracy: 0.9803 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 0.0223 - val_binary_accuracy: 1.0000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00


In [13]:
watch_model.save('xception_multiclass1.h5')

Unfreeze a few more layers and allow to run

In [ ]:
epochs = 50

In [24]:
watch_model.layers[0].summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Conv2D)            (None, None, None, 32 864         input_1[0][0]                    
____________________________________________________________________________________________________
block1_conv1_bn (BatchNormalizat (None, None, None, 32 128         block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_conv1_act (Activation)    (None, None, None, 32 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [26]:
# Freeze convolutional layers
for layer in watch_model.layers[0].layers[-3:]:
    layer.trainable = True

In [30]:
epochs = 50

In [31]:
# fine-tune the model
watch_model.fit_generator(
        multiclass_flow_train(train_generator, multiclasses_getter),
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data = multiclass_flow_test(test_generator, multiclasses_getter),
        validation_steps=nb_validation_samples // batch_size)

Epoch 1/50
 21/921 [..............................] - ETA: 299s - loss: 1.1387 - binary_accuracy: 0.7091 - precision: 0.0000e+00 - recall: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
watch_model.save('xception_multiclass_unfrozen_convos1.h5')